### After cleaning the initial dataset, we are going to look at enriching it with a key variable which is going to be used in some models as the predictor variable, which is company valuation

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
pd.set_option("display.max_columns", None)

In [2]:
startups = pd.read_csv("../../dataset/Cleaned_startups.csv", index_col=0)
url_dr = "https://app.dealroom.co/companies/"

In [3]:
startups.head()

,permalink,name,market,funding_total_usd,status,country_code,city,funding_rounds,founded_at,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,delta_founded_vs_firstfunding
0,/organization/waywire,#waywire,News,1750000,acquired,USA,New York,1.0,2012-06-01,2012.0,2012-06-30,2012-06-30,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0
3,/organization/in-touch-network,(In)Touch Network,Electronics,1500000,operating,GBR,London,1.0,2011-04-01,2011.0,2011-04-01,2011-04-01,1500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,Tourism,60000,operating,USA,Fort Worth,2.0,2014-01-01,2014.0,2014-08-17,2014-09-26,0.0,0.0,60000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,228.0
8,/organization/004-technologies,004 Technologies,Software,0,operating,USA,Champaign,1.0,2010-01-01,2010.0,2014-07-24,2014-07-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1665.0
10,/organization/1-2-3-listo,"1,2,3 Listo",E-Commerce,40000,operating,CHL,Las Condes,1.0,2012-01-01,2012.0,2013-02-18,2013-02-18,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,414.0


### We will get the valuation from Deal Room webpage, I will set a for loop, but will do an individual try first

In [4]:
from selenium import webdriver
from time import sleep

In [5]:
driver = webdriver.Chrome("../../tools/Selenium/chromedriver.exe")

In [6]:
driver.get("https://app.dealroom.co/companies")
sleep(2)

In [7]:
from selenium.webdriver.common.keys import Keys

In [8]:
driver.find_elements_by_css_selector("header div div div div div input")[0].send_keys("Field Nation")

In [9]:
driver.find_elements_by_css_selector("header div div div div div input")[0].send_keys(u'\ue007')

In [12]:
driver.find_elements_by_css_selector("div ul li a div h3")[0].click()

In [13]:
driver.find_elements_by_css_selector("span.valuation__value")[0].text

'€109—164m'

In [14]:
driver.close()

### Now we will do the loop to get this value for every company

In [ ]:
startups_valuation = []
for i, row in startups.iterrows():
    try:
        name = row["name"]
        driver = webdriver.Chrome("../../tools/Selenium/chromedriver.exe")
        driver.get("https://app.dealroom.co/companies")
        sleep(1)
        driver.find_elements_by_css_selector("header div div div div div input")[0].send_keys(name)
        sleep(1)
        driver.find_elements_by_css_selector("header div div div div div input")[0].send_keys(u'\ue007')
        sleep(2)
        driver.find_elements_by_css_selector("div ul li a div h3")[0].click()
        startups_valuation.append(driver.find_elements_by_css_selector("span.valuation__value")[0].text)
        driver.close()
        sleep(3)
    except:
        startups_valuation.append(0)

### I have had trouble with Selenium as the web has detected it's a robot scraping so I will be using Beatiful Soup instead.  However, the code for the loop would look like the above

### First, let's define some parsers we will use for the web browser links when including the company name at the end of it

In [15]:
def parser1(name):
    return name.lower().replace(" ", "")

In [16]:
parser1("My Startup")

'mystartup'

In [17]:
def parser2(name):
    return name.lower().replace(" ", "-")

In [18]:
parser2("My Startup")

'my-startup'

In [19]:
def parser3(name):
    return name.lower().replace(" ", "_")

In [20]:
parser3("My Startup")

'my_startup'

In [21]:
parsers = [parser1, parser2, parser3]

In [22]:
def obtain_valuation_parser(name, parser):
    url = url_dr + parser(name)

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    company_funding = soup.find(name="span", class_="valuation__value").text
    
    return company_funding

In [23]:
obtain_valuation_parser("Acclaimd", parser1)

'€255—382k'

### Let's make it so that the function can run through the 3 parsers and look for the valuation

In [24]:
def obtain_valuation(name):
    # tries all parsers in the list until one works. Then prints info and returns valuation
    # else returns None and prints Nope
    for p in parsers:
        try:
            val = obtain_valuation_parser(name, p)
            print(f"{name:30} used {p.__name__:10} valuation: {val}")
            return val
        except:
            continue
    else:
        print(f"{name:30} {'Nope':10}")

In [25]:
obtain_valuation("My Startup")

My Startup                     Nope      


In [26]:
obtain_valuation("Acclaimd")

Acclaimd                       used parser1    valuation: €255—382k


'€255—382k'

In [27]:
obtain_valuation("Bee Cave Games")

Bee Cave Games                 used parser3    valuation: €20—30m


'€20—30m'

### Let's apply the function to the whole dataset now

In [28]:
import numpy as np

In [29]:
startups["valuation"] = np.nan
for i, s in startups.iterrows():
    v = obtain_valuation(s["name"])
    startups.loc[i, "valuation"] = v
    if i % 10 == 0:
        startups.to_csv("startups_with_valuation_temp.csv")
        print(f"saved at index {i}")

#waywire                       Nope      
saved at index 0
(In)Touch Network              Nope      
-R- Ranch and Mine             Nope      
004 Technologies               Nope      
1,2,3 Listo                    Nope      
saved at index 10
1-800-DENTIST                  Nope      
1-800-DOCTORS                  Nope      
1.618 Technology               Nope      
10 Minutes With                used parser3    valuation: €14—22m
10-20 Media                    Nope      
1000 Corks                     used parser3    valuation: €145—218k
1000museums.com                Nope      
1001 Menus                     Nope      
1006.tv                        Nope      
100health                      Nope      
100Plus                        used parser1    valuation: €91—136m
1010data                       used parser1    valuation: €140—210m
10�North                       Nope      
10BestThings                   used parser1    valuation: €182—273k
saved at index 30
10seconds Software    

3Pillar Global                 Nope      
3Play Media                    Nope      
3ROAM                          used parser1    valuation: €4—7m
3scale                         used parser1    valuation: €3—4m
3Scan                          used parser1    valuation: €24—36m
saved at index 250
3seventy                       Nope      
3Sourcing                      Nope      
3TEN8                          Nope      
3TIER                          used parser1    valuation: €13m
3Touch                         used parser1    valuation: €982k—1m
3V Transaction Services        used parser3    valuation: €15m
3VR                            used parser1    valuation: €29—44m
3X Systems                     used parser3    valuation: €6—9m
saved at index 260
4 the stars                    used parser3    valuation: €4—5k
4-Tell                         Nope      
405Labs                        Nope      
40billion.com                  Nope      
410 Labs                       used parser3  

AAIPharma Services             used parser3    valuation: €190m
AAMPP                          Nope      
Aarden Pharmaceuticals         used parser3    valuation: €2—3m
Aaron Andrews Apparel          Nope      
Aasonn                         used parser1    valuation: €127—182m
Aastrom Biosciences            Nope      
aaTag                          used parser1    valuation: €722k—1m
saved at index 510
Aava Mobile                    used parser3    valuation: €15—23m
AB Tasty                       used parser3    valuation: €145—218m
ab&jb properties and services  Nope      
ABA English                    used parser3    valuation: €44—65m
Abacast                        used parser1    valuation: €2—3m
saved at index 520
Abacus e-Media                 Nope      
Abacus Labs                    Nope      
Abakan                         Nope      
Abakus                         used parser1    valuation: €3—5m
ABARIS Investment Management   Nope      
Abaxia                         used

Accord Biomaterials            used parser3    valuation: €4—7m
Accounting SaaS Japan          Nope      
AccountNow                     used parser1    valuation: €47—70m
Accruent                       used parser1    valuation: €1.8b
Accruit                        used parser1    valuation: €12—18m
Accu-Break Pharmaceuticals     Nope      
saved at index 710
Accudial Pharmaceutical        Nope      
AccuDraft                      used parser1    valuation: €2—4m
Accuhealth Partners            Nope      
AccuMED Technologies           Nope      
Accumetrics                    used parser1    valuation: €38—55m
Accumulate                     used parser1    valuation: €36—55m
Accumuli Security              used parser3    valuation: €24—36m
AccuNostics                    used parser1    valuation: €19—29m
Accupal                        Nope      
Accuradio                      used parser1    valuation: €9—14m
Accurence                      used parser1    valuation: €9—13m
AccuRev    

ActualMeds                     used parser1    valuation: €4—7m
ActuatedMedical                used parser1    valuation: €545—818k
Acturis                        Nope      
Actus Digital                  used parser3    valuation: €7—11m
Actus Interactive Software     Nope      
ACTV8me                        Nope      
ActX                           used parser1    valuation: €3—4m
Acucar Guarani                 used parser3    valuation: €165—248m
Acucela                        used parser1    valuation: €2—2m
saved at index 910
AcuFocus                       used parser1    valuation: €76—115m
Acuity Medical International   used parser3    valuation: €5—8m
AcuityAds                      used parser1    valuation: €545m
Acumatica                      used parser1    valuation: €91—136m
Acumen Holdings                used parser3    valuation: €528—755m
Acumen Pharmaceuticals         used parser3    valuation: €73—109m
Acumentrics                    Nope      
saved at index 920
Acun

...

Yapper                         Nope      
Yappn                          Nope      
YapStone                       used parser1    valuation: €428m
Yapta                          used parser1    valuation: €7—10m
YapTime                        used parser1    valuation: €727k—1m
Yard Club                      used parser3    valuation: €6—9m
Yarraa                         used parser1    valuation: €5—8m
Yashi                          used parser1    valuation: €30m
saved at index 48440
Yassets                        Nope      
YASSSU                         used parser1    valuation: €2—3m
Yasuu                          Nope      
Yatango Mobile                 used parser3    valuation: €5—8m
Yatedo                         used parser1    valuation: €8—12m
Yatown                         used parser1    valuation: €1—2m
saved at index 48450
Yava Technologies              used parser3    valuation: €16—24m
yavalu                         Nope      
Ybrain                         used pa

YouDocs Beauty                 Nope      
YouDroop LTD                   used parser3    valuation: €360—540k
YouEye                         used parser1    valuation: €12—18m
YouFetch                       Nope      
saved at index 48670
YouGift                        used parser1    valuation: €4—7m
YouGoDo                        used parser1    valuation: €400—600k
YouGotListings                 used parser1    valuation: €545—818k
YouGov                         used parser1    valuation: €1.6b
YouHelp                        Nope      
YouLicense                     used parser1    valuation: €4—5m
saved at index 48680
Youlicit                       Nope      
YouLike                        Nope      
YouMail                        Nope      
Youmiam                        used parser1    valuation: €4—7m
Youneeq                        used parser1    valuation: €2—3m
Young Innovations              Nope      
YoungCracks                    Nope      
YOUnite                        u

Zaranga                        Nope      
Zarbee's                       Nope      
Zarpamos.com                   Nope      
Zarpo                          Nope      
Zartis                         used parser1    valuation: €1—2m
Zattikka                       used parser1    valuation: €72—108m
Zattoo                         Nope      
saved at index 48900
Zauber                         used parser1    valuation: €4—5m
Zawatt                         used parser1    valuation: €7—11m
Zaya                           Nope      
Zazom                          used parser1    valuation: €109—164k
Zazoom                         Nope      
Zazuba                         used parser1    valuation: €545—818k
Zazzle                         used parser1    valuation: €109—164m
Zazzy                          Nope      
Z�m XR                         Nope      
ZBD Displays                   used parser3    valuation: €20—29m
Zdorovio                       used parser1    valuation: €364—545k
Ze 

Ziegler                        Nope      
Zientia                        Nope      
Zift Solutions                 used parser3    valuation: €7—10m
Ziften Technologies            used parser3    valuation: €53—80m
saved at index 49130
Zify                           used parser1    valuation: €7m
Zigabid                        used parser1    valuation: €2—3m
ZigaVite                       Nope      
Zighra                         used parser1    valuation: €4—5m
Zigmo                          Nope      
Zignal Labs                    used parser3    valuation: €109—164m
Zignals                        used parser1    valuation: €10—15m
saved at index 49140
Zigswitch                      used parser1    valuation: €182—273k
ZIIBRA                         used parser1    valuation: €429—644k
Ziios                          used parser1    valuation: €3—5m
Ziipa                          Nope      
Ziippi                         Nope      
Zikk Software Ltd.             Nope      
Ziklag Sy

Zounds                         used parser1    valuation: €36—55m
Zoutons                        used parser1    valuation: €12—18m
Zoyi                           used parser1    valuation: €15—23m
ZOZI                           used parser1    valuation: €36—55m
saved at index 49340
zPerfectGift                   Nope      
ZPower                         Nope      
ZS Genetics                    used parser3    valuation: €6—9m
ZS Pharma                      used parser3    valuation: €200—300m
Zscaler                        used parser1    valuation: €27.5b
zSoup                          used parser1    valuation: €327—491k
Ztail                          used parser1    valuation: €4—6m
Ztory                          Nope      
saved at index 49350
Zuberance                      used parser1    valuation: €29—44m
Zubican                        Nope      
Zubie                          used parser1    valuation: €29—44m
ZUCHEM                         Nope      
Zuga Medical           

### Let's import the updated csv with valuations

In [88]:
startups = pd.read_csv("./startups_with_valuation_temp.csv", index_col=0)

In [89]:
startups.head()

,permalink,name,market,funding_total_usd,status,country_code,city,funding_rounds,founded_at,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,delta_founded_vs_firstfunding,valuation
0,/organization/waywire,#waywire,News,1750000,acquired,USA,New York,1.0,2012-06-01,2012.0,2012-06-30,2012-06-30,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,NaN
3,/organization/in-touch-network,(In)Touch Network,Electronics,1500000,operating,GBR,London,1.0,2011-04-01,2011.0,2011-04-01,2011-04-01,1500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,Tourism,60000,operating,USA,Fort Worth,2.0,2014-01-01,2014.0,2014-08-17,2014-09-26,0.0,0.0,60000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,228.0,NaN
8,/organization/004-technologies,004 Technologies,Software,0,operating,USA,Champaign,1.0,2010-01-01,2010.0,2014-07-24,2014-07-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1665.0,NaN
10,/organization/1-2-3-listo,"1,2,3 Listo",E-Commerce,40000,operating,CHL,Las Condes,1.0,2012-01-01,2012.0,2013-02-18,2013-02-18,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,414.0,NaN


### We can see that in some cases no valuation has been extracted. Let's delete those rows and leave only the ones without Nones

In [90]:
startups = startups[startups["valuation"].notnull()]

In [91]:
startups.shape

(19809, 34)

### Let's now clean the valuation values

In [92]:
def valuation_cleaner(x):
    if "—" in x:
        a = x.split("—")
        return a[1]
    else:
        return x

In [93]:
valuation_cleaner("€4.4b")

'€4.4b'

In [94]:
startups["valuation"] = startups["valuation"].apply(valuation_cleaner) 

### Given that we want all the values to be consistent let's show all valuations in millions

In [95]:
def integer_converter(x):
    if "k" in x:
        valuation = x.replace("k","")
        valuation_clean = valuation.replace("€","")
        return float(valuation_clean)/1000
    elif "m" in x:
        valuation = x.replace("m","")
        valuation_clean = valuation.replace("€","")
        return round(float(valuation_clean),1)
    elif "b" in x:
        valuation = x.replace("b","")
        valuation_clean = valuation.replace("€","")
        return round(float(valuation_clean)*1000,1)

In [96]:
startups["valuation"] = startups["valuation"].apply(integer_converter) 

In [97]:
startups.head()

,permalink,name,market,funding_total_usd,status,country_code,city,funding_rounds,founded_at,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,delta_founded_vs_firstfunding,valuation
15,/organization/10-minutes-with,10 Minutes With,Education,4400000,operating,GBR,London,2.0,2013-01-01,2013.0,2013-01-01,2014-10-09,400000.0,4000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.000
17,/organization/1000-corks,1000 Corks,Search,40000,operating,USA,Lake Oswego,1.0,2008-01-01,2008.0,2011-08-23,2011-08-23,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1330.0,0.218
27,/organization/100plus,100Plus,Analytics,1250000,acquired,USA,San Francisco,2.0,2011-09-16,2011.0,2011-11-02,2011-11-30,750000.0,500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.0,136.000
28,/organization/1010data,1010data,Software,35000000,operating,USA,New York,1.0,2000-01-01,2000.0,2010-03-08,2010-03-08,0.0,35000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3719.0,210.000
30,/organization/10bestthings,10BestThings,Curated Web,50000,closed,USA,Cleveland,1.0,2009-04-01,2009.0,2009-04-01,2009-04-01,50000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.273


In [98]:
startups.to_csv("startups_with_valuation_v_final.csv")